<div id='id0' > </div>

<body>
    <h1> <b> Análisis Exploratorio de Datos </b> </h1> 
    <h2> <b> Índice </b> </h2>
</body>

1.  [Generalidades](#id1)
2.  [Características Preliminares de las Fuentes de Datos](#id2)
3.  [Imoportación de la Librerías Requeridas](#id3)
4.  [Extración de los Datos](#id4)
5.  [Carga de los Datos en Dataframes](#id5)
6.  [Fuente de Datos: Títulos de Amazon Prime](#id6)
7.  [Fuente de Datos: Títulos de Disney Plus](#id7)
8.  [Fuente de Datos: Títulos de Hulu](#id8)
9.  [Fuente de Datos: Títulos de Hulu](#id9)
10. [Unión de los Dataframes](#id10)
11. [Obtención de los Insights](#id11)

---


<div id='id1' > </div>

<body>
     <h1> <b> Generalidades </b> <br> </h1>
     <p> Los datos fueron provistos en archivos de diferentes extensiones, como csv o json. La información ha de ser procesada y normalizada para cada dataset, entre otras tareas, para obtener información relevante, tal como: <p>
     <ul>
          <li> <p> 
          Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])</b>. </li>
          <li> <p> 
          Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)</b>. </li>
          <li> <p> 
          Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')</b>. </li>
          <li> <p> 
          Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)</b>. </li>
     </ul> 
</body>




[ Índice </p> ](#id0)

---

<div id='id2' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Características Preliminares de las Fuentes de Datos </b> <br> </h1>
     <p>
     Los datasets disponibles corresponden ha los titulos disponibles en cuatro proveedores de streaming. <br>
     <br>
     Los recursos recuperados de las fuentes fueron:
          <ul>
               <li> <p> 
               Títulos de Amazon Prime: amazon_prime_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Disney Plus: disney_plus_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Hulu: hulu_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Netflix: netflix_titles.json</b>. </li>
               </ul> 
     </p>
</body>

[ Índice </p> ](#id0)

---

<div id='id3' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Imoportación de la Librerías Requeridas </b> <br> </h1>
</body>

In [1]:
#Importar las librerías
#Python 
import os
from pathlib import Path
from datetime import datetime

#Request
import requests

#Pandas
import pandas as pd

[ Índice </p> ](#id0)

---

<div id='id4' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Extración de los Datos </b> <br> </h1>
</body>

In [2]:
#Definir los directorios y las rutas
MAIN_DIR = (Path(os.path.abspath(''))).parent                                    #out: "$PWD/streaming_catalog_api"
DATA_DIR = Path(f"{MAIN_DIR}/data")                                              #out: "$PWD/streaming_catalog_api/data"
EXT_DIR =  Path(f"{DATA_DIR}/external")                                          #out: "$PWD/streaming_catalog_api/data/external"
INT_DIR = Path(f"{DATA_DIR}/interim")                                            #out: "$PWD/streaming_catalog_api/data/interim"
PRO_DIR = Path(f"{MAIN_DIR}/processed")                                          #out: "$PWD/streaming_catalog_api/data/raw"

In [ ]:
print(
    "MAIN_DIR:" f"{MAIN_DIR}\n",
    "DATA_DIR:" f"{DATA_DIR}\n",
    "EXT_DIR:" f"{EXT_DIR}\n",
    "INT_DIR:" f"{INT_DIR}\n",
    "PRO_DIR:" f"{PRO_DIR}\n"
)

In [ ]:
#Crear una lista de diccionarios para almacenar la categoría y la url
category_list = [
    {
        "name": "amazon_prime_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/amazon_prime_titles.csv"
    },
    {
        "name": "disney_plus_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/disney_plus_titles.csv"
    },
    {
        "name": "hulu_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/hulu_titles.csv"
    },
    {
        "name": "netflix_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/netflix_titles.json"
    }
    ]

In [ ]:
#Definir la función get_data: esta función obtiene mediante web scraping las fuentes de datos
def get_data(category_list):
    for i in range(len(category_list)):
        streaming_plataform = category_list[i]["name"]
        if streaming_plataform != "netflix_titles":
            file_name = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv"
        else:
            file_name = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.json"
        
        now = datetime.now()
        category = category_list[i]["name"]
        year = now.year
        month = now.month
        day = now.day
        
        r = requests.get(category_list[i]["url"])
        r.encoding = 'utf-8'
        FILE_NAME = file_name.format(
            category = category, 
            year = year, 
            month = month, 
            day = day
            )

        FILE_PATH = EXT_DIR/FILE_NAME
        FILE_PATH.parent.mkdir(
            parents=True,
            exist_ok=True
            )
            
        with open(FILE_PATH,'w') as file_outp:
            file_outp.write(r.text)
            print(f"Se ha guardado el archivo en el directorio: {FILE_PATH} \n")

In [ ]:
#Ejecutar la función get data
get_data(category_list = category_list)

[ Índice </p> ](#id0)

---

<div id='id5' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Carga de los Datos en Dataframes </b> <br> </h1>
</body>

<div id='id6' > </div>

<body>
     <h2> <b> Fuente de Datos</b>: Títulos de amazon prime </h2>
</body>

In [3]:
#Declarar los parámetros para la carga del dataframe
path = "c:/Users/Jose Salas/OneDrive/Escritorio/streaming_catalog_api/data/external/amazon_prime_titles/2022-12/amazon_prime_titles-08-12-2022.csv"
sep = ","
cod = "utf-8"

amz_df = pd.read_csv(filepath_or_buffer = path, delimiter = sep, encoding = cod)

In [4]:
#Visualizar preliminarmente el dataframe
amz_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [5]:
#Inspeccionar el dataframe
amz_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
release_year,9668.0,2008.341849,18.922482,1920.0,2007.0,2016.0,2019.0,2021.0


In [6]:
# Identificar variables categóricas y numércias/ Identifie cateorical and numerical vars
num_vars = amz_df.columns[amz_df.dtypes != "object"]
cat_vars = amz_df.columns[amz_df.dtypes == "object"]

print("Variables numericas:")
for var in num_vars:
    print(f"\t{var}")
print("\n")

print("Variables categóricas:")
for var in cat_vars:
    print(f"\t{var}")

Variables numericas:
	release_year


Variables categóricas:
	show_id
	type
	title
	director
	cast
	country
	date_added
	rating
	duration
	listed_in
	description


In [7]:
# Visualizar las columnas del dataframe y sus tipos de datos
amz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [8]:
#IIdentificar registros nulos
amz_df.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

In [9]:
#Iddentificar registros vacíos
amz_df.isna().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

<body>
     <h3> <b> Selección de las Columnas de Interes </b> </h3>
</body>

In [59]:
#Definir las columnas que deseamos conservar
columns = [ 
            "show_id", 
            "type", 
            "title", 
            "cast", 
            "date_added",
            "release_year", 
            "duration", 
            "listed_in"
            ]

In [60]:
#Inspeccionar preliminarmente el dataframe reducido
amz_df[columns].head()

,show_id,type,title,cast,date_added,release_year,duration,listed_in
0,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent","March 30, 2021",2014,113 min,"Comedy, Drama"
1,s2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar","March 30, 2021",2018,110 min,"Drama, International"
2,s3,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...","March 30, 2021",2017,74 min,"Action, Drama, Suspense"
3,s4,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...","March 30, 2021",2014,69 min,Documentary
4,s5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...","March 30, 2021",1989,45 min,"Drama, Fantasy"


<body>
     <h3> <b> Inspección de datos nulos por columna de interés </b> </h3>
</body>

In [12]:
#Datos nulos, columna "cast"
amz_df.loc[amz_df["cast"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
20,s21,TV Show,Zoboomafoo,NaN,NaN,NaN,NaN,2001,TV-Y,1 Season,Kids,Chris and Martin Kratt bring their enthusiasm ...
25,s26,Movie,Zambezi: Force of Life,Dr. Rudolf Lammers,NaN,NaN,NaN,2005,ALL,50 min,Documentary,The Zambezi river is the fourth-longest river ...
33,s34,Movie,Young Love (at the Sun Factory),NaN,NaN,NaN,NaN,2007,NR,36 min,Music Videos and Concerts,"Young Love celebrates the release of ""Too Youn..."
34,s35,Movie,Young Love (at Bowery Ballroom),NaN,NaN,NaN,NaN,2007,NR,37 min,Music Videos and Concerts,Young Love returns to Baeble's silver screen w...
70,s71,Movie,World Inferno Friendship Society,Baeble Music,NaN,NaN,NaN,2007,NR,40 min,Music Videos and Concerts,The circus comes to town with this performance...
...,...,...,...,...,...,...,...,...,...,...,...,...
9636,s9637,Movie,Pieces of a Dream: A Story of Gambling,Michelle Wong,NaN,NaN,NaN,2003,13+,49 min,"Documentary, Special Interest","When Phillip Wong died by suicide, his family ..."
9637,s9638,TV Show,Paul Newman Trilogy,NaN,NaN,NaN,NaN,2021,13+,1 Season,TV Shows,The early days of television were filled with ...
9645,s9646,TV Show,Let's Go Cozy Coupe! - Cartoons for Kids,NaN,NaN,NaN,NaN,2021,ALL,1 Season,Kids,Cozy Coupe is an online series that helps kids...
9647,s9648,TV Show,Kids In The Hall,NaN,NaN,NaN,"October 10, 2021",1995,TV-14,5 Seasons,"Arts, Entertainment, and Culture, Comedy","""The Kids in the Hall"" is a unique and bizarre..."


In [13]:
#Datos nulos, columna "date_added"
amz_df.loc[amz_df["date_added"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
16,s17,Movie,Zoombies,Glenn Miller,"Marcus Anderson, Kaiwi Lyman, Andrew Asper",NaN,NaN,2016,13+,87 min,"Horror, Science Fiction",When a strange virus quickly spreads through a...
17,s18,TV Show,Zoo Babies,NaN,Narrator - Gillian Barlett,NaN,NaN,2008,ALL,1 Season,"Kids, Special Interest",A heart warming and inspiring series that welc...
18,s19,TV Show,Zoë Coombs Marr: Bossy Bottom,NaN,Zoë Coombs Marr,NaN,NaN,2020,18+,1 Season,"Comedy, Talk Show and Variety",Zoë Coombs Marr has been on hiatus. Sort of. F...
19,s20,Movie,Zoe,Drake Doremus,"Ewan McGregor, Léa Seydoux, Theo James, Mirand...",NaN,NaN,2018,R,104 min,Science Fiction,ZOE tells a tale of forbidden love between an ...
20,s21,TV Show,Zoboomafoo,NaN,NaN,NaN,NaN,2001,TV-Y,1 Season,Kids,Chris and Martin Kratt bring their enthusiasm ...
...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve..."
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."


<body>
     <h3> <b> Transformación del dataframe </b> </h3>
</body>

In [61]:
#Duplicar el dataframe original conservando únicamente las columnas de interes
amz_edit_df = amz_df[columns]

In [62]:
#   Definimos la nueva denominación que tendrán las columnas de interes
renamed_columns = {
                    "show_id":"id_show",
                    "type":"tipo",
                    "title":"titulo",
                    "cast":"elenco",
                    "date_added":"fecha_añadidura",
                    "release_year":"estreno",
                    "duration":"duracion",
                    "listed_in":"genero_cinematrografico"
                    }

In [63]:
#Renombrar el dataframe
amz_edit_df.rename(columns = renamed_columns, inplace = True)

C:\Users\Jose Salas\AppData\Local\Temp\ipykernel_24544\691127811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amz_edit_df.rename(columns = renamed_columns, inplace = True)


In [64]:
#Comprobar que el dataframe haya sido actualizado con los cambios indicados
amz_edit_df.head()

,id_show,tipo,titulo,elenco,fecha_añadidura,estreno,duracion,genero_cinematrografico
0,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent","March 30, 2021",2014,113 min,"Comedy, Drama"
1,s2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar","March 30, 2021",2018,110 min,"Drama, International"
2,s3,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...","March 30, 2021",2017,74 min,"Action, Drama, Suspense"
3,s4,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...","March 30, 2021",2014,69 min,Documentary
4,s5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...","March 30, 2021",1989,45 min,"Drama, Fantasy"


In [65]:
#Añadir tipo de plataforma
amz_edit_df["plataforma"] = "Amazon Prime"

C:\Users\Jose Salas\AppData\Local\Temp\ipykernel_24544\646495300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amz_edit_df["plataforma"] = "Amazon Prime"


In [66]:
#Inspeccionar el df
amz_edit_df.head()

,id_show,tipo,titulo,elenco,fecha_añadidura,estreno,duracion,genero_cinematrografico,plataforma
0,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent","March 30, 2021",2014,113 min,"Comedy, Drama",Amazon Prime
1,s2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar","March 30, 2021",2018,110 min,"Drama, International",Amazon Prime
2,s3,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...","March 30, 2021",2017,74 min,"Action, Drama, Suspense",Amazon Prime
3,s4,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...","March 30, 2021",2014,69 min,Documentary,Amazon Prime
4,s5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...","March 30, 2021",1989,45 min,"Drama, Fantasy",Amazon Prime


In [72]:
#Dar de baja lógica a los datos nulos
amz_edit_df["estatus"] = 1
amz_edit_df["estatus"].loc[amz_df["cast"].isnull()] = 0

C:\Users\Jose Salas\AppData\Local\Temp\ipykernel_24544\816854927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amz_edit_df["estatus"] = 1
C:\Users\Jose Salas\AppData\Local\Temp\ipykernel_24544\816854927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amz_edit_df["estatus"].loc[amz_df["cast"].isnull()] = 0


In [75]:
amz_edit_df.loc[amz_df["cast"].isnull()].head()

,id_show,tipo,titulo,elenco,fecha_añadidura,estreno,duracion,genero_cinematrografico,plataforma,estatus
20,s21,TV Show,Zoboomafoo,NaN,NaN,2001,1 Season,Kids,Amazon Prime,0
25,s26,Movie,Zambezi: Force of Life,NaN,NaN,2005,50 min,Documentary,Amazon Prime,0
33,s34,Movie,Young Love (at the Sun Factory),NaN,NaN,2007,36 min,Music Videos and Concerts,Amazon Prime,0
34,s35,Movie,Young Love (at Bowery Ballroom),NaN,NaN,2007,37 min,Music Videos and Concerts,Amazon Prime,0
70,s71,Movie,World Inferno Friendship Society,NaN,NaN,2007,40 min,Music Videos and Concerts,Amazon Prime,0


[ Índice </p> ](#id0)

---

<div id='id7' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h2> <b> Fuente de Datos</b>: Títulos de Disney Plus </h2>
</body>

In [14]:
# Declarar los parámetros para la carga del dataframe
path = "c:/Users/Jose Salas/OneDrive/Escritorio/streaming_catalog_api/data/external/disney_plus_titles/2022-12/disney_plus_titles-08-12-2022.csv"
sep = ","
cod = "utf-8"

dsny_df = pd.read_csv(filepath_or_buffer=path, delimiter=sep, encoding=cod)


In [15]:
#Visualizar preliminarmente el dataframe
dsny_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [16]:
#Inspeccionar el dataframe
dsny_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
release_year,1450.0,2003.091724,21.860162,1928.0,1999.0,2011.0,2018.0,2021.0


In [17]:
# Identificar variables categóricas y numércias/ Identifie cateorical and numerical vars
num_vars = dsny_df.columns[dsny_df.dtypes != "object"]
cat_vars = dsny_df.columns[dsny_df.dtypes == "object"]

print("Variables numericas:")
for var in num_vars:
    print(f"\t{var}")
print("\n")

print("Variables categóricas:")
for var in cat_vars:
    print(f"\t{var}")

Variables numericas:
	release_year


Variables categóricas:
	show_id
	type
	title
	director
	cast
	country
	date_added
	rating
	duration
	listed_in
	description


In [18]:
# Visualizar las columnas del dataframe y sus tipos de datos
dsny_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [19]:
#IIdentificar registros nulos
dsny_df.isnull().sum()

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64

In [20]:
#Iddentificar registros vacíos
dsny_df.isna().sum()

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64

<body>
     <h3> <b> Selección de las Columnas de Interes </b> </h3>
</body>

In [ ]:
#Definir las columnas que deseamos conservar
columns = [ 
            "show_id", 
            "type", 
            "title", 
            "cast", 
            "date_added", 
            "duration", 
            "listed_in"
            ]

In [21]:
#Inspeccionar preliminarmente el dataframe reducido
dsny_df[columns].head()

,show_id,type,title,cast,date_added,duration,listed_in
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Chris Diamantopoulos, Tony Anselmo, Tress MacN...","November 26, 2021",23 min,"Animation, Family"
1,s2,Movie,Ernest Saves Christmas,"Jim Varney, Noelle Parker, Douglas Seale","November 26, 2021",91 min,Comedy
2,s3,Movie,Ice Age: A Mammoth Christmas,"Raymond Albert Romano, John Leguizamo, Denis L...","November 26, 2021",23 min,"Animation, Comedy, Family"
3,s4,Movie,The Queen Family Singalong,"Darren Criss, Adam Lambert, Derek Hough, Alexa...","November 26, 2021",41 min,Musical
4,s5,TV Show,The Beatles: Get Back,"John Lennon, Paul McCartney, George Harrison, ...","November 25, 2021",1 Season,"Docuseries, Historical, Music"


<body>
     <h3> <b> Inspección de datos nulos por columna de interés </b> </h3>
</body>

In [22]:
#Datos nulos, columna "cast"
dsny_df.loc[dsny_df["cast"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
11,s12,Movie,Puppy for Hanukkah,NaN,NaN,NaN,"November 19, 2021",2020,TV-G,4 min,"Comedy, Family, Music","Check out Daveed Digg's official music video ""..."
15,s16,Movie,Baymax!,NaN,NaN,NaN,"November 12, 2021",2021,TV-G,1 min,Animation,“Baymax!” premieres on Disney+ summer 2022.
26,s27,Movie,Marvel Studios’ 2021 Disney+ Day Special,NaN,NaN,NaN,"November 12, 2021",2021,TV-PG,14 min,Movies,Marvel Studios’ Disney+ Day Special explores t...
27,s28,Movie,Obi-Wan Kenobi,NaN,NaN,NaN,"November 12, 2021",2021,TV-PG,1 min,"Action-Adventure, Science Fiction","A behind the scenes look of the ""Obi-Wan Kenob..."
35,s36,Movie,The Little Matchgirl,Roger Allers,NaN,United States,"November 12, 2021",2006,G,7 min,"Animation, Drama",A poor matchgirl sees happy images in the flam...
...,...,...,...,...,...,...,...,...,...,...,...,...
1363,s1364,Movie,Three Little Wolves,Dave Hand,NaN,United States,"November 12, 2019",1936,TV-Y7,9 min,"Animation, Family",Two frivolous pigs blow the wolf horn one too ...
1375,s1376,Movie,Toby Tortoise Returns,Wilfred Jackson,NaN,United States,"November 12, 2019",1936,TV-Y7,8 min,"Animation, Family, Sports",Max Hare and Toby Tortoise are opponents in a ...
1384,s1385,Movie,Trailer Horn,Jack Hannah,NaN,United States,"November 12, 2019",1950,TV-G,7 min,"Animation, Comedy, Family",Donald heads out for a nice relaxing camping t...
1411,s1412,Movie,Winter Storage,Jack Hannah,NaN,United States,"November 12, 2019",1949,TV-G,7 min,"Animation, Comedy, Family",Donald and Chip and Dale face off over acorns....


In [23]:
#Datos nulos, columna "date_added"
dsny_df.loc[dsny_df["date_added"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1439,s1440,TV Show,Disney Kirby Buckets,NaN,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,NaN,2014,TV-Y7,3 Seasons,"Action-Adventure, Comedy, Coming of Age",Welcome to Kirby's world! It's rude and sketchy.
1440,s1441,TV Show,Disney Mech-X4,NaN,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,NaN,2016,TV-Y7,2 Seasons,"Action-Adventure, Comedy, Science Fiction",Ryan discovers his ability to control a giant ...
1441,s1442,TV Show,Imagination Movers,NaN,"Rich Collins, Dave Poche, Scott Durbin, Scott ...",United States,NaN,2008,TV-Y,3 Seasons,"Kids, Music","Rock out with the Imagination Movers, Disney J..."


<body>
     <h3> <b> Transformación del dataframe </b> </h3>
</body>

In [ ]:
#Duplicar el dataframe original conservando únicamente las columnas de interes
dsny_edit_df = dsny_df[columns]

[ Índice </p> ](#id0)

---

<div id='id6' > </div>

<body>
     <h2> <b> Fuente de Datos</b>: Títulos de hulu </h2>
</body>

In [24]:
#Declarar los parámetros para la carga del dataframe
path = "c:/Users/Jose Salas/OneDrive/Escritorio/streaming_catalog_api/data/external/hulu_titles/2022-12/hulu_titles-08-12-2022.csv"
sep = ","
cod = "utf-8"

hl_df = pd.read_csv(filepath_or_buffer = path, delimiter = sep, encoding = cod)

In [25]:
#Visualizar preliminarmente el dataframe
hl_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [26]:
#Inspeccionar el dataframe
hl_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
cast,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
release_year,3073.0,2012.567524,10.844069,1923.0,2010.0,2016.0,2019.0,2021.0


In [27]:
# Identificar variables categóricas y numércias/ Identifie cateorical and numerical vars
num_vars = hl_df.columns[hl_df.dtypes != "object"]
cat_vars = hl_df.columns[hl_df.dtypes == "object"]

print("Variables numericas:")
for var in num_vars:
    print(f"\t{var}")
print("\n")

print("Variables categóricas:")
for var in cat_vars:
    print(f"\t{var}")

Variables numericas:
	cast
	release_year


Variables categóricas:
	show_id
	type
	title
	director
	country
	date_added
	rating
	duration
	listed_in
	description


In [28]:
# Visualizar las columnas del dataframe y sus tipos de datos
hl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [29]:
#IIdentificar registros nulos
hl_df.isnull().sum()

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64

In [30]:
#Iddentificar registros vacíos
hl_df.isna().sum()

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64

<body>
     <h3> <b> Selección de las Columnas de Interes </b> </h3>
</body>

In [31]:
#Definir las columnas que deseamos conservar
columns = [ 
            "show_id", 
            "type", 
            "title", 
            "cast", 
            "date_added", 
            "duration", 
            "listed_in"
            ]

In [32]:
#Inspeccionar preliminarmente el dataframe reducido
amz_df[columns].head()

,show_id,type,title,cast,date_added,duration,listed_in
0,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent","March 30, 2021",113 min,"Comedy, Drama"
1,s2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar","March 30, 2021",110 min,"Drama, International"
2,s3,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...","March 30, 2021",74 min,"Action, Drama, Suspense"
3,s4,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...","March 30, 2021",69 min,Documentary
4,s5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...","March 30, 2021",45 min,"Drama, Fantasy"


<body>
     <h3> <b> Selección de las Columnas de Interes </b> </h3>
</body>

In [33]:
#Inspeccionar preliminarmente el dataframe reducido
hl_df[columns].head()

,show_id,type,title,cast,date_added,duration,listed_in
0,s1,Movie,Ricky Velez: Here's Everything,NaN,"October 24, 2021",NaN,"Comedy, Stand Up"
1,s2,Movie,Silent Night,NaN,"October 23, 2021",94 min,"Crime, Drama, Thriller"
2,s3,Movie,The Marksman,NaN,"October 23, 2021",108 min,"Action, Thriller"
3,s4,Movie,Gaia,NaN,"October 22, 2021",97 min,Horror
4,s5,Movie,Settlers,NaN,"October 22, 2021",104 min,"Science Fiction, Thriller"


<body>
     <h3> <b> Inspección de datos nulos por columna de interés </b> </h3>
</body>

In [35]:
#Datos nulos, columna "cast"
hl_df.loc[hl_df["cast"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...
...,...,...,...,...,...,...,...,...,...,...,...,...
3068,s3069,TV Show,Star Trek: The Original Series,NaN,NaN,United States,NaN,1966,TV-PG,3 Seasons,"Action, Adventure, Classics",The 23rd century adventures of Captain James T...
3069,s3070,TV Show,Star Trek: Voyager,NaN,NaN,United States,NaN,1997,TV-PG,7 Seasons,"Action, Adventure, Science Fiction",Catapulted into the distant sector of the gala...
3070,s3071,TV Show,The Fades,NaN,NaN,United Kingdom,NaN,2011,TV-14,1 Season,"Horror, International, Science Fiction",Seventeen-year-old Paul is haunted by apocalyp...
3071,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...


In [36]:
#Datos nulos, columna "date_added"
hl_df.loc[hl_df["date_added"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3045,s3046,TV Show,Angel,NaN,NaN,United States,NaN,1999,TV-14,5 Seasons,"Action, Adventure, Drama","Joss Whedon, the creator and executive produce..."
3046,s3047,TV Show,Arrested Development,NaN,NaN,United States,NaN,2003,TV-14,3 Seasons,"Comedy, Sitcom","In this critically acclaimed series, Michael B..."
3047,s3048,TV Show,Black Butler: Book of Circus,NaN,NaN,Japan,NaN,2014,TV-14,1 Season,"Action, Adventure, Anime",When children begin to vanish after the arriva...
3048,s3049,TV Show,Blade Dance of the Elementalers,NaN,NaN,Japan,NaN,2014,TV-14,1 Season,"Action, Adventure, Anime",The prestigious Areishia Spirit Academy has tr...
3049,s3050,TV Show,Boys Before Flowers,NaN,NaN,South Korea,NaN,2009,TV-14,1 Season,"Comedy, Drama, International",Boys Over Flowers is one of the biggest Korean...
3050,s3051,TV Show,Buffy the Vampire Slayer,NaN,NaN,United States,NaN,1997,TV-PG,7 Seasons,"Drama, Science Fiction, Teen",Buffy the Vampire Slayer is a comedy-action se...
3051,s3052,TV Show,Doctora Juguetes,NaN,NaN,NaN,NaN,2012,TV-Y,1 Season,"Kids, Latino",Una mágica serie animada acerca de Doc McStuff...
3052,s3053,TV Show,Firefly,NaN,NaN,United States,NaN,2002,TV-PG,1 Season,"Action, Adventure, Science Fiction",The Crew of a starship travel through outer sp...
3053,s3054,TV Show,Frasier,NaN,NaN,United States,NaN,1993,TV-PG,11 Seasons,"Comedy, Sitcom","Comedy series set in Seattle, WA, which chroni..."
3054,s3055,TV Show,Hey Arnold!,NaN,NaN,United States,NaN,1996,TV-Y,5 Seasons,"Cartoons, Comedy, Family",Football-headed Arnold lives with his offbeat ...


In [37]:
#Datos nulos, columna "duration"
hl_df.loc[hl_df["duration"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
12,s13,Movie,Out of the Shadows: The Man Behind the Steele ...,NaN,NaN,NaN,"October 18, 2021",2021,NaN,NaN,News,George Stephanopoulos sits down with former MI...
15,s16,Movie,Showtime Championship Boxing: Lopez vs. Salido...,NaN,NaN,NaN,"October 16, 2021",2011,NaN,NaN,Sports,Juan Manuel Lopez vs. Orlando Salido for the W...
37,s38,Movie,BELLATOR MMA: Kongo vs. Johnson 2,NaN,NaN,NaN,"October 13, 2021",2020,NaN,NaN,Sports,Cheick Kongo vs. Tim Johnson battle in a histo...
40,s41,Movie,In the Crease,NaN,NaN,NaN,"October 13, 2021",2020,NaN,NaN,Sports,Post-game analysis and highlight show airing e...
...,...,...,...,...,...,...,...,...,...,...,...,...
2951,s2952,Movie,All Saints (1998),NaN,NaN,Australia,"February 19, 2014",1998,12 Seasons,NaN,"Drama, International",All Saints is an Australian medical drama focu...
2955,s2956,Movie,Getting On,NaN,NaN,United Kingdom,"January 7, 2014",2009,3 Seasons,NaN,"Comedy, International, Sitcom",Care for the elderly is the least glamorous ar...
2958,s2959,Movie,(Sub) Ninja Scroll,NaN,NaN,Japan,"December 15, 2013",1993,93 min,NaN,"Action, Adult Animation, Adventure",When Jubei saves a young ninja woman from the ...
2959,s2960,Movie,Packed to the Rafters,NaN,NaN,Australia,"December 3, 2013",2008,6 Seasons,NaN,"Comedy, Drama, International",Packed to the Rafters is an Australian family-...


<body>
     <h3> <b> Transformación del dataframe </b> </h3>
</body>

In [ ]:
#Duplicar el dataframe original conservando únicamente las columnas de interes
hl_edit_df = hl_df[columns]

[ Índice </p> ](#id0)

---

<div id='id7' > </div>

<body>
     <h2> <b> Fuente de Datos</b>: Títulos de netflix </h2>
</body>

In [42]:
#Declarar los parámetros para la carga del dataframe
path = "c:/Users/Jose Salas/OneDrive/Escritorio/streaming_catalog_api/data/external/netflix_titles/2022-12/netflix_titles-08-12-2022.json"
sep = ","
cod = "utf-8"

ntflx_df = pd.read_json(path)

In [43]:
#Visualizar preliminarmente el dataframe
ntflx_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [44]:
#Inspeccionar el dataframe
ntflx_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
release_year,8807.0,2014.180198,8.819312,1925.0,2013.0,2017.0,2019.0,2021.0


In [45]:
# Identificar variables categóricas y numércias/ Identifie cateorical and numerical vars
num_vars = ntflx_df.columns[ntflx_df.dtypes != "object"]
cat_vars = ntflx_df.columns[ntflx_df.dtypes == "object"]

print("Variables numericas:")
for var in num_vars:
    print(f"\t{var}")
print("\n")

print("Variables categóricas:")
for var in cat_vars:
    print(f"\t{var}")

Variables numericas:
	release_year


Variables categóricas:
	show_id
	type
	title
	director
	cast
	country
	date_added
	rating
	duration
	listed_in
	description


In [46]:
# Visualizar las columnas del dataframe y sus tipos de datos
ntflx_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB


In [47]:
#IIdentificar registros nulos
ntflx_df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [48]:
#Iddentificar registros vacíos
ntflx_df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

<body>
     <h3> <b> Selección de las Columnas de Interes </b> </h3>
</body>

In [ ]:
#Definir las columnas que deseamos conservar
columns = [ 
            "show_id", 
            "type", 
            "title", 
            "cast", 
            "date_added", 
            "duration", 
            "listed_in"
            ]

In [49]:
#Inspeccionar preliminarmente el dataframe reducido
ntflx_df[columns].head()

,show_id,type,title,cast,date_added,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,None,"September 25, 2021",90 min,Documentaries
1,s2,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","September 24, 2021",2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
2,s3,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","September 24, 2021",1 Season,"Crime TV Shows, International TV Shows, TV Act..."
3,s4,TV Show,Jailbirds New Orleans,None,"September 24, 2021",1 Season,"Docuseries, Reality TV"
4,s5,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","September 24, 2021",2 Seasons,"International TV Shows, Romantic TV Shows, TV ..."


<body>
     <h3> <b> Inspección de datos nulos por columna de interés </b> </h3>
</body>

In [50]:
#Datos nulos, columna "cast"
ntflx_df.loc[ntflx_df["cast"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
10,s11,TV Show,"Vendetta: Truth, Lies and The Mafia",None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, Docuseries, International TV S...","Sicily boasts a bold ""Anti-Mafia"" coalition. B..."
14,s15,TV Show,Crime Stories: India Detectives,None,None,None,"September 22, 2021",2021,TV-MA,1 Season,"British TV Shows, Crime TV Shows, Docuseries",Cameras following Bengaluru police on the job ...
16,s17,Movie,Europe's Most Dangerous Man: Otto Skorzeny in ...,"Pedro de Echave García, Pablo Azorín Williams",None,None,"September 22, 2021",2020,TV-MA,67 min,"Documentaries, International Movies",Declassified documents reveal the post-WWII li...
...,...,...,...,...,...,...,...,...,...,...,...,...
8746,s8747,Movie,Winnie,Pascale Lamche,None,"France, Netherlands, South Africa, Finland","February 26, 2018",2017,TV-14,85 min,"Documentaries, International Movies",Winnie Mandela speaks about her extraordinary ...
8755,s8756,TV Show,Women Behind Bars,None,None,United States,"November 1, 2016",2010,TV-14,3 Seasons,"Crime TV Shows, Docuseries",This reality series recounts true stories of w...
8756,s8757,Movie,Woodstock,Barak Goodman,None,United States,"August 13, 2019",2019,TV-MA,97 min,"Documentaries, Music & Musicals",For the 50th anniversary of the legendary Wood...
8763,s8764,Movie,WWII: Report from the Aleutians,John Huston,None,United States,"March 31, 2017",1943,TV-PG,45 min,Documentaries,Filmmaker John Huston narrates this Oscar-nomi...


In [51]:
#Datos nulos, columna "date_added"
ntflx_df.loc[ntflx_df["date_added"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6066,s6067,TV Show,A Young Doctor's Notebook and Other Stories,None,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,None,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."
6174,s6175,TV Show,Anthony Bourdain: Parts Unknown,None,Anthony Bourdain,United States,None,2018,TV-PG,5 Seasons,Docuseries,This CNN original series has chef Anthony Bour...
6795,s6796,TV Show,Frasier,None,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,None,2003,TV-PG,11 Seasons,"Classic & Cult TV, TV Comedies",Frasier Crane is a snooty but lovable Seattle ...
6806,s6807,TV Show,Friends,None,"Jennifer Aniston, Courteney Cox, Lisa Kudrow, ...",United States,None,2003,TV-14,10 Seasons,"Classic & Cult TV, TV Comedies",This hit sitcom follows the merry misadventure...
6901,s6902,TV Show,Gunslinger Girl,None,"Yuuka Nanri, Kanako Mitsuhashi, Eri Sendai, Am...",Japan,None,2008,TV-14,2 Seasons,"Anime Series, Crime TV Shows","On the surface, the Social Welfare Agency appe..."
7196,s7197,TV Show,Kikoriki,None,Igor Dmitriev,None,None,2010,TV-Y,2 Seasons,Kids' TV,A wacky rabbit and his gang of animal pals hav...
7254,s7255,TV Show,La Familia P. Luche,None,"Eugenio Derbez, Consuelo Duval, Luis Manuel Áv...",United States,None,2012,TV-14,3 Seasons,"International TV Shows, Spanish-Language TV Sh...","This irreverent sitcom featues Ludovico, Feder..."
7406,s7407,TV Show,Maron,None,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,None,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
7847,s7848,TV Show,Red vs. Blue,None,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,None,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
8182,s8183,TV Show,The Adventures of Figaro Pho,None,"Luke Jurevicius, Craig Behenna, Charlotte Haml...",Australia,None,2015,TV-Y7,2 Seasons,"Kids' TV, TV Comedies","Imagine your worst fears, then multiply them: ..."


In [52]:
#Datos nulos, columna "date_added"
ntflx_df.loc[ntflx_df["duration"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,74 min,None,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,84 min,None,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,66 min,None,Movies,The comic puts his trademark hilarious/thought...


<body>
     <h3> <b> Transformación del dataframe </b> </h3>
</body>

In [ ]:
#Duplicar el dataframe original conservando únicamente las columnas de interes
ntflx_edit_df = ntflx_df[columns]